# Exercice 2 : Jointures

1 - Chargement des données
2 - Jointure LEFT
3 - Calcul aggrégé
4 - Enrichissement
5 - Export & validation

In [16]:
# 1 - Chargement des données :
import pandas as pd
import os
import numpy as np
from pathlib import Path   

CUSTOMERS_FILE_PATH = Path('../../../data_src/clients2.csv')
TRANSACTIONS_FILE_PATH = Path('../../../data_src/transactions.csv')    


customers_df = pd.read_csv(CUSTOMERS_FILE_PATH)
transactions_df = pd.read_csv(TRANSACTIONS_FILE_PATH)


display(customers_df.head()) 
display(transactions_df.head())


,client_id,nom,ville,age
0,1,Client_1,Toulouse,60
1,2,Client_2,Paris,23
2,3,Client_3,Nice,61
3,4,Client_4,Bordeaux,57
4,5,Client_5,Lyon,47


,transaction_id,client_id,date_transaction,montant
0,1,9,2025-01-01,128.78
1,2,10,2025-01-08,477.10
2,3,9,2025-01-15,381.29
3,4,4,2025-01-22,234.17
4,5,18,2025-01-29,219.39


In [17]:
# Jointure LEFT:
# Un fichier client, un fichier transactions
# Joindre les transactions aux clients, on transactions.client_id = clients.client_id

joined_df = pd.merge(customers_df, transactions_df, how='left', left_on='client_id', right_on='client_id', suffixes=('_client', '_transaction'))
display(joined_df.head(5))

,client_id,nom,ville,age,transaction_id,date_transaction,montant
0,1,Client_1,Toulouse,60,34,2025-08-20,57.95
1,2,Client_2,Paris,23,21,2025-05-21,397.24
2,2,Client_2,Paris,23,39,2025-09-24,396.80
3,3,Client_3,Nice,61,24,2025-06-11,329.11
4,4,Client_4,Bordeaux,57,4,2025-01-22,234.17


In [ ]:
# 3 - Calcul aggrégé:
# Métrique: total_amount_per_client (somme des montants de transactions)
total_amount_per_client = (
	joined_df
	.groupby('client_id')['montant']
	.sum()
	.reset_index()
	.rename(columns={'montant': 'sum_orders_amount'})
)
# Trier par montant décroissant et afficher les 5 premiers
total_amount_per_client = total_amount_per_client.sort_values(by='sum_orders_amount', ascending=False)


# Nombre de commandes par client
nombre_commandes = (
	joined_df
	.groupby('client_id')['montant']
	.count()
	.reset_index()
	.rename(columns={'montant': 'num_orders'})
)
nombre_commandes = nombre_commandes.sort_values(by='num_orders', ascending=False)

panier_moyen = (
	total_amount_per_client
	.merge(nombre_commandes, on='client_id')
)
panier_moyen['average_basket'] = panier_moyen['sum_orders_amount'] / panier_moyen['num_orders']
panier_moyen = panier_moyen[['client_id', 'average_basket']].sort_values(by='average_basket', ascending=False)

metrics_df = total_amount_per_client.merge(panier_moyen, on='client_id').merge(nombre_commandes, on='client_id')
display(metrics_df.head(5))

,client_id,sum_orders_amount,average_basket,num_orders
0,9,2121.14,265.142500,8
1,6,1805.30,300.883333,6
2,14,1428.62,357.155000,4
3,20,1075.58,358.526667,3
4,18,1061.89,265.472500,4


In [25]:
# 4 - Enrichissement des données clients:
# Ajouter la colonne total_amount_per_client au dataframe customers_df
customers_enriched_df = pd.merge(customers_df, metrics_df, how='left', left_on='client_id', right_on='client_id')
# Remplacer les valeurs NaN par 0 dans la colonne sum_orders_amount
customers_enriched_df['sum_orders_amount'] = customers_enriched_df['sum_orders_amount'].fillna(0)
customers_enriched_df['num_orders'] = customers_enriched_df['num_orders'].fillna(0)
customers_enriched_df['average_basket'] = customers_enriched_df['average_basket'].fillna(0)

customers_enriched_df['num_orders'] = customers_enriched_df['num_orders'].astype(int)
customers_enriched_df['average_basket'] = customers_enriched_df['average_basket'].astype(float)
customers_enriched_df['sum_orders_amount'] = customers_enriched_df['sum_orders_amount'].astype(float)

customers_enriched_df = customers_enriched_df.sort_values(by='sum_orders_amount', ascending=False)

display(customers_enriched_df.head(5))

,client_id,nom,ville,age,sum_orders_amount,average_basket,num_orders
8,9,Client_9,Lyon,50,2121.14,265.142500,8
5,6,Client_6,Paris,48,1805.30,300.883333,6
13,14,Client_14,Marseille,28,1428.62,357.155000,4
19,20,Client_20,Bordeaux,35,1075.58,358.526667,3
17,18,Client_18,Toulouse,36,1061.89,265.472500,4


In [20]:
# 5 - Export & validation
OUTPUT_FILE_PATH = Path('../../../data_cleaned/customers_enriched.csv')
customers_enriched_df.to_csv(OUTPUT_FILE_PATH, index=False) 
# Recharger et afficher
customers_enriched_reloaded_df = pd.read_csv(OUTPUT_FILE_PATH)
display(customers_enriched_reloaded_df.head(5))

,client_id,nom,ville,age,sum_orders_amount
0,1,Client_1,Toulouse,60,57.95
1,2,Client_2,Paris,23,794.04
2,3,Client_3,Nice,61,329.11
3,4,Client_4,Bordeaux,57,905.72
4,5,Client_5,Lyon,47,449.45
